<a href="https://colab.research.google.com/github/Codehackerone/nlp-with-transformers/blob/main/Open_Domain_QA_with_HayStack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests

In [2]:
data = requests.get('https://raw.githubusercontent.com/jamescalam/transformers/main/data/text/meditations/clean.txt')
text = data.text.split('\n')


In [ ]:
requests.get('http://localhost:9200/_cluster/health').json()

In [ ]:
print(requests.get('http://localhost:9200/_cat/indices').text)

In [ ]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='aurelius'
)

In [ ]:
print(requests.get('http://localhost:9200/_cat/indices').text)

In [ ]:
data_json = [
    {
        'text': paragraph,
        'meta': {
            'source': 'meditations'
        }
    } for paragraph in text
]

In [ ]:
doc_store.write_documents(data_json)

In [ ]:
requests.get('http://localhost:9200/aurelius/_count').json()

In [ ]:
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

doc_store = ElasticsearchDocumentStore(
    host='localhost',
    username='', password='',
    index='aurelius'
)

In [ ]:
from haystack.retriever.sparse import ElasticsearchRetriever
from haystack.reader.farm import FARMReader

retriever = ElasticsearchRetriever(doc_store)  # BM25
reader = FARMReader(model_name_or_path='deepset/bert-base-cased-squad2',
                    context_window_size=1500,
                    use_gpu=True)

In [ ]:
from haystack.pipeline import ExtractiveQAPipeline

qa = ExtractiveQAPipeline(reader=reader, retriever=retriever)

In [ ]:
qa.run(query='What did your grandfather teach?')

In [ ]:
qa.run(query='What is the Universe?',
       top_k_reader=3)

In [ ]:
qa.run(query='What is art?',
       top_k_reader=3)